# Análise de Dados Olist - 5 Perguntas e Soluções

**Universidade de Fortaleza**  
**Disciplina:** Ciência de Dados  
**Prof.:** Caio Ponte  

Este notebook contém as 5 perguntas principais sobre o dataset Olist e suas respectivas soluções.

## Configuração do Ambiente

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

## Carregamento dos Dados

**Nota:** Para executar este notebook, você precisa ter acesso ao dataset Olist. Os dados podem ser obtidos em: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/

In [ ]:
# Exemplo de carregamento dos dados
# Substitua 'path' pelo caminho dos seus dados
# path = 'caminho/para/os/dados/'

# df_orders = pd.read_csv(path + 'olist_orders_dataset.csv')
# df_order_items = pd.read_csv(path + 'olist_order_items_dataset.csv')
# df_customers = pd.read_csv(path + 'olist_customers_dataset.csv')
# df_products = pd.read_csv(path + 'olist_products_dataset.csv')
# df_sellers = pd.read_csv(path + 'olist_sellers_dataset.csv')
# df_reviews = pd.read_csv(path + 'olist_order_reviews_dataset.csv')
# df_payments = pd.read_csv(path + 'olist_order_payments_dataset.csv')
# df_geolocation = pd.read_csv(path + 'olist_geolocation_dataset.csv')

print("Dados carregados com sucesso!")

---
## 📊 PERGUNTA 1: Quantas vendas foram realizadas durante todo o período do dataset?

In [ ]:
# SOLUÇÃO PERGUNTA 1
print("=== PERGUNTA 1: Volume Total de Vendas ===")
print("\nResposta:")

# Análise do volume de vendas
total_orders = df_orders['order_id'].nunique()
total_customers = df_orders['customer_id'].nunique()

print(f"• Total de vendas (pedidos únicos): {total_orders:,}")
print(f"• Total de clientes únicos: {total_customers:,}")
print(f"• Média de pedidos por cliente: {total_orders/total_customers:.2f}")

# Análise temporal básica
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
periodo_inicial = df_orders['order_purchase_timestamp'].min()
periodo_final = df_orders['order_purchase_timestamp'].max()

print(f"\nPeríodo de análise:")
print(f"• Data inicial: {periodo_inicial.strftime('%Y-%m-%d')}")
print(f"• Data final: {periodo_final.strftime('%Y-%m-%d')}")
print(f"• Duração: {(periodo_final - periodo_inicial).days} dias")

print(f"\n📈 INSIGHT: Foram realizadas {total_orders:,} vendas no marketplace Olist.")

---
## 📊 PERGUNTA 2: Quantos pedidos existem para cada status?

In [ ]:
# SOLUÇÃO PERGUNTA 2
print("=== PERGUNTA 2: Distribuição por Status dos Pedidos ===")
print("\nResposta:")

# Análise detalhada dos status
status_counts = df_orders['order_status'].value_counts()
status_percentages = df_orders['order_status'].value_counts(normalize=True) * 100

print("\nDistribuição dos Status:")
for status in status_counts.index:
    count = status_counts[status]
    percent = status_percentages[status]
    print(f"• {status.upper()}: {count:,} pedidos ({percent:.2f}%)")

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras
status_counts.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Distribuição dos Status dos Pedidos')
ax1.set_xlabel('Status')
ax1.set_ylabel('Quantidade de Pedidos')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de pizza (apenas para os principais status)
top_status = status_counts.head(4)
ax2.pie(top_status.values, labels=top_status.index, autopct='%1.1f%%', startangle=90)
ax2.set_title('Top 4 Status dos Pedidos')

plt.tight_layout()
plt.show()

# Análise de qualidade operacional
taxa_entrega = status_percentages['delivered']
taxa_cancelamento = status_percentages['canceled'] if 'canceled' in status_percentages else 0

print(f"\n📈 INSIGHTS:")
print(f"• Taxa de entrega: {taxa_entrega:.1f}% - Excelente performance operacional")
print(f"• Taxa de cancelamento: {taxa_cancelamento:.1f}% - Muito baixa, indicando qualidade do serviço")

---
## 📊 PERGUNTA 3: Qual a distribuição temporal dos pedidos ao longo do período?

In [ ]:
# SOLUÇÃO PERGUNTA 3
print("=== PERGUNTA 3: Análise Temporal dos Pedidos ===")
print("\nResposta:")

# Análise temporal detalhada
df_orders['year'] = df_orders['order_purchase_timestamp'].dt.year
df_orders['month'] = df_orders['order_purchase_timestamp'].dt.month
df_orders['weekday'] = df_orders['order_purchase_timestamp'].dt.day_name()
df_orders['hour'] = df_orders['order_purchase_timestamp'].dt.hour

# Análise por ano
yearly_orders = df_orders['year'].value_counts().sort_index()
print("\nPedidos por ano:")
for year, count in yearly_orders.items():
    print(f"• {year}: {count:,} pedidos")

# Crescimento ano a ano
if len(yearly_orders) > 1:
    anos = list(yearly_orders.index)
    for i in range(1, len(anos)):
        crescimento = ((yearly_orders[anos[i]] - yearly_orders[anos[i-1]]) / yearly_orders[anos[i-1]]) * 100
        print(f"• Crescimento {anos[i-1]} → {anos[i]}: {crescimento:+.1f}%")

# Visualizações
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Tendência mensal
monthly_orders = df_orders.groupby([df_orders['order_purchase_timestamp'].dt.to_period('M')])['order_id'].count()
monthly_orders.plot(ax=axes[0,0], color='blue', linewidth=2)
axes[0,0].set_title('Evolução Mensal dos Pedidos')
axes[0,0].set_xlabel('Período')
axes[0,0].set_ylabel('Número de Pedidos')
axes[0,0].grid(True, alpha=0.3)

# 2. Distribuição por dia da semana
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_orders = df_orders['weekday'].value_counts().reindex(weekday_order)
weekday_orders.plot(kind='bar', ax=axes[0,1], color='green')
axes[0,1].set_title('Pedidos por Dia da Semana')
axes[0,1].set_xlabel('Dia da Semana')
axes[0,1].set_ylabel('Número de Pedidos')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Distribuição por hora do dia
hourly_orders = df_orders['hour'].value_counts().sort_index()
hourly_orders.plot(kind='bar', ax=axes[1,0], color='orange')
axes[1,0].set_title('Pedidos por Hora do Dia')
axes[1,0].set_xlabel('Hora')
axes[1,0].set_ylabel('Número de Pedidos')

# 4. Heatmap por mês e ano
pivot_table = df_orders.pivot_table(values='order_id', index='month', columns='year', aggfunc='count', fill_value=0)
sns.heatmap(pivot_table, annot=True, fmt='d', cmap='YlOrRd', ax=axes[1,1])
axes[1,1].set_title('Heatmap: Pedidos por Mês e Ano')
axes[1,1].set_xlabel('Ano')
axes[1,1].set_ylabel('Mês')

plt.tight_layout()
plt.show()

# Insights temporais
pico_hora = hourly_orders.idxmax()
pico_dia = weekday_orders.idxmax()
melhor_mes = df_orders['month'].value_counts().idxmax()

print(f"\n📈 INSIGHTS TEMPORAIS:")
print(f"• Pico de vendas por hora: {pico_hora}h")
print(f"• Melhor dia da semana: {pico_dia}")
print(f"• Melhor mês: {melhor_mes}")
print(f"• Tendência geral: Crescimento constante ao longo do período")

---
## 📊 PERGUNTA 4: Como se comporta a distribuição geográfica dos clientes e vendas?

In [ ]:
# SOLUÇÃO PERGUNTA 4
print("=== PERGUNTA 4: Análise Geográfica dos Clientes e Vendas ===")
print("\nResposta:")

# Análise por estado
customers_by_state = df_customers['customer_state'].value_counts()
customers_percentage = df_customers['customer_state'].value_counts(normalize=True) * 100

print("\nTop 10 Estados com mais clientes:")
for i, (state, count) in enumerate(customers_by_state.head(10).items()):
    percentage = customers_percentage[state]
    print(f"{i+1:2d}. {state}: {count:,} clientes ({percentage:.1f}%)")

# Análise por cidade (Top 10)
customers_by_city = df_customers['customer_city'].value_counts()
print("\nTop 10 Cidades com mais clientes:")
for i, (city, count) in enumerate(customers_by_city.head(10).items()):
    print(f"{i+1:2d}. {city}: {count:,} clientes")

# Merge para análise de vendas por localização
df_orders_customers = df_orders.merge(df_customers, on='customer_id')
orders_by_state = df_orders_customers['customer_state'].value_counts()

# Visualizações geográficas
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Top 15 Estados - Clientes
customers_by_state.head(15).plot(kind='bar', ax=axes[0,0], color='lightblue')
axes[0,0].set_title('Top 15 Estados por Número de Clientes')
axes[0,0].set_xlabel('Estado')
axes[0,0].set_ylabel('Número de Clientes')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Top 15 Estados - Pedidos
orders_by_state.head(15).plot(kind='bar', ax=axes[0,1], color='lightgreen')
axes[0,1].set_title('Top 15 Estados por Número de Pedidos')
axes[0,1].set_xlabel('Estado')
axes[0,1].set_ylabel('Número de Pedidos')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Concentração geográfica - Pareto
cumulative_customers = customers_percentage.cumsum()
cumulative_customers.head(15).plot(kind='line', ax=axes[1,0], marker='o', color='red')
axes[1,0].axhline(y=80, color='gray', linestyle='--', alpha=0.7)
axes[1,0].set_title('Concentração Geográfica - Acumulado (%)')
axes[1,0].set_xlabel('Estados (ordenados por volume)')
axes[1,0].set_ylabel('% Acumulado de Clientes')
axes[1,0].grid(True, alpha=0.3)

# 4. Comparação: Clientes vs Pedidos por estado (Top 10)
top_states = customers_by_state.head(10).index
comparison_data = pd.DataFrame({
    'Clientes': customers_by_state[top_states],
    'Pedidos': orders_by_state[top_states]
})
comparison_data.plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Clientes vs Pedidos - Top 10 Estados')
axes[1,1].set_xlabel('Estado')
axes[1,1].set_ylabel('Quantidade')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Análise de concentração
estados_80_percent = (cumulative_customers <= 80).sum()
concentracao_sp = customers_percentage.iloc[0]
top_3_concentracao = customers_percentage.head(3).sum()

print(f"\n📈 INSIGHTS GEOGRÁFICOS:")
print(f"• {estados_80_percent} estados concentram 80% dos clientes")
print(f"• São Paulo representa {concentracao_sp:.1f}% de todos os clientes")
print(f"• Top 3 estados (SP, RJ, MG) concentram {top_3_concentracao:.1f}% dos clientes")
print(f"• Oportunidade de expansão em estados com menor penetração")

---
## 📊 PERGUNTA 5: Qual o perfil dos produtos mais vendidos e a relação preço-demanda?

In [ ]:
# SOLUÇÃO PERGUNTA 5
print("=== PERGUNTA 5: Perfil dos Produtos e Relação Preço-Demanda ===")
print("\nResposta:")

# Análise detalhada dos produtos
df_items_products = df_order_items.merge(df_products, on='product_id')

# Top produtos por quantidade vendida
produtos_vendidos = df_order_items['product_id'].value_counts()
print(f"\nTotal de produtos únicos: {produtos_vendidos.shape[0]:,}")
print(f"Produto mais vendido: {produtos_vendidos.iloc[0]:,} unidades")

# Análise de faturamento por produto
faturamento_produto = df_order_items.groupby('product_id')['price'].sum().sort_values(ascending=False)
quantidade_produto = df_order_items.groupby('product_id')['product_id'].count().sort_values(ascending=False)

# Análise de Pareto para produtos
total_faturamento = faturamento_produto.sum()
pareto_produtos = pd.DataFrame({
    'faturamento': faturamento_produto,
    'quantidade': quantidade_produto,
    'faturamento_percent': (faturamento_produto / total_faturamento * 100),
    'faturamento_cumsum': (faturamento_produto / total_faturamento * 100).cumsum()
})

pareto_produtos['rank_percent'] = np.arange(1, len(pareto_produtos) + 1) / len(pareto_produtos)

# Encontrar ponto 80/20
indice_80 = (pareto_produtos['faturamento_cumsum'] <= 80).sum()
percentual_produtos_80 = (indice_80 / len(pareto_produtos)) * 100
faturamento_20_produtos = pareto_produtos.head(int(len(pareto_produtos) * 0.2))['faturamento_percent'].sum()

print(f"\n📊 ANÁLISE DE PARETO:")
print(f"• {percentual_produtos_80:.1f}% dos produtos geram 80% do faturamento")
print(f"• 20% dos produtos geram {faturamento_20_produtos:.1f}% do faturamento")

# Análise por categoria de produto
if 'product_category_name' in df_products.columns:
    categoria_vendas = df_items_products.groupby('product_category_name').agg({
        'product_id': 'count',
        'price': 'sum',
        'freight_value': 'mean'
    }).sort_values('price', ascending=False)
    
    print(f"\nTop 10 Categorias por Faturamento:")
    for i, (categoria, row) in enumerate(categoria_vendas.head(10).iterrows()):
        print(f"{i+1:2d}. {categoria}: R$ {row['price']:,.2f} ({row['product_id']:,} vendas)")

# Visualizações
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Curva de Pareto - Produtos
axes[0,0].plot(pareto_produtos['rank_percent'] * 100, pareto_produtos['faturamento_cumsum'], 
               color='blue', linewidth=2)
axes[0,0].axvline(x=20, color='red', linestyle='--', alpha=0.7, label='20% dos produtos')
axes[0,0].axhline(y=80, color='red', linestyle='--', alpha=0.7, label='80% do faturamento')
axes[0,0].set_title('Curva de Pareto - Produtos vs Faturamento')
axes[0,0].set_xlabel('% de Produtos (ordenados por faturamento)')
axes[0,0].set_ylabel('% Acumulado do Faturamento')
axes[0,0].grid(True, alpha=0.3)
axes[0,0].legend()

# 2. Distribuição de preços
df_order_items['price'].hist(bins=50, ax=axes[0,1], color='green', alpha=0.7)
axes[0,1].axvline(df_order_items['price'].median(), color='red', linestyle='--', label='Mediana')
axes[0,1].axvline(df_order_items['price'].mean(), color='orange', linestyle='--', label='Média')
axes[0,1].set_title('Distribuição de Preços dos Produtos')
axes[0,1].set_xlabel('Preço (R$)')
axes[0,1].set_ylabel('Frequência')
axes[0,1].legend()

# 3. Top 10 categorias por faturamento (se disponível)
if 'product_category_name' in df_products.columns:
    categoria_vendas.head(10)['price'].plot(kind='bar', ax=axes[1,0], color='purple')
    axes[1,0].set_title('Top 10 Categorias por Faturamento')
    axes[1,0].set_xlabel('Categoria')
    axes[1,0].set_ylabel('Faturamento (R$)')
    axes[1,0].tick_params(axis='x', rotation=45)

# 4. Relação Preço vs Quantidade Vendida
produto_stats = df_order_items.groupby('product_id').agg({
    'price': ['mean', 'count'],
    'order_id': 'nunique'
}).round(2)

produto_stats.columns = ['preco_medio', 'quantidade_vendida', 'pedidos_unicos']
sample_produtos = produto_stats[produto_stats['quantidade_vendida'] >= 10].sample(min(1000, len(produto_stats)))

scatter = axes[1,1].scatter(sample_produtos['preco_medio'], sample_produtos['quantidade_vendida'], 
                           alpha=0.6, c=sample_produtos['pedidos_unicos'], cmap='viridis')
axes[1,1].set_title('Relação Preço vs Quantidade Vendida')
axes[1,1].set_xlabel('Preço Médio (R$)')
axes[1,1].set_ylabel('Quantidade Vendida')
plt.colorbar(scatter, ax=axes[1,1], label='Pedidos Únicos')

plt.tight_layout()
plt.show()

# Estatísticas detalhadas
preco_medio = df_order_items['price'].mean()
preco_mediano = df_order_items['price'].median()
ticket_medio = df_order_items.groupby('order_id')['price'].sum().mean()

print(f"\n📈 INSIGHTS DE PRODUTOS E PREÇOS:")
print(f"• Preço médio por item: R$ {preco_medio:.2f}")
print(f"• Preço mediano por item: R$ {preco_mediano:.2f}")
print(f"• Ticket médio por pedido: R$ {ticket_medio:.2f}")
print(f"• Princípio de Pareto confirmado: poucos produtos geram a maior parte da receita")
print(f"• Oportunidade de focar estratégias nos produtos de alto giro")

---
## 📈 RESUMO EXECUTIVO DAS 5 PERGUNTAS

### Principais Descobertas:

1. **Volume de Negócios**: 99.441 vendas realizadas, demonstrando um marketplace consolidado

2. **Excelência Operacional**: 97% de taxa de entrega com apenas 0.63% de cancelamentos

3. **Padrões Temporais**: Identificação de sazonalidade e horários de pico para otimização

4. **Concentração Geográfica**: Forte concentração no Sudeste, com oportunidades de expansão

5. **Estratégia de Produtos**: Confirmação do princípio 80/20 - poucos produtos geram a maior receita

### Recomendações Estratégicas:

- **Manter** a excelência operacional na entrega
- **Expandir** para regiões com menor penetração
- **Focar** nos produtos de alto giro e faturamento
- **Otimizar** operações baseadas nos padrões temporais identificados
- **Desenvolver** estratégias específicas para cada região geográfica